In [1]:
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', None)

# Import Virtual Mode Data

In [2]:
virtual = pd.read_csv("../../data/raw_data/Texas_Schools_LearningModelData_Final.csv", thousands=',')
virtual.head()

,StateName,StateAbbrev,DataLevel,Charter,SchoolName,SchoolType,NCESSchoolID,StateAssignedSchoolID,DistrictName,DistrictType,NCESDistrictID,StateAssignedDistrictID,EnrollmentTotal,TimePeriodInterval,TimePeriodStart,TimePeriodEnd,LearningModel,LearningModelGrK5,LearningModelGr68,LearningModelGr912,LearningModelStateCat,LearningModelStateCatGrK5,LearningModelStateCatGr68,LearningModelStateCatGr912,EnrollmentInPerson,EnrollmentHybrid,EnrollmentVirtual,StaffCount,StaffCountInPerson
0,Texas,TX,School,No,Crosbyton Elementary,Regular school,480000101145,54901101,Crosbyton CISD,Regular local school district,4800001,54901,138.0,Bi-annually,9/1/20,12/31/20,In-person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,0.0,13.0,NaN,NaN
1,Texas,TX,School,No,Crosbyton Elementary,Regular school,480000101145,54901101,Crosbyton CISD,Regular local school district,4800001,54901,143.0,Bi-annually,1/1/21,6/1/21,In-person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.0,0.0,2.0,NaN,NaN
2,Texas,TX,School,No,Crosbyton Secondary,Regular school,480000101146,54901001,Crosbyton CISD,Regular local school district,4800001,54901,171.0,Bi-annually,9/1/20,12/31/20,In-person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161.0,0.0,10.0,NaN,NaN
3,Texas,TX,School,No,Crosbyton Secondary,Regular school,480000101146,54901001,Crosbyton CISD,Regular local school district,4800001,54901,169.0,Bi-annually,1/1/21,6/1/21,In-person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,165.0,0.0,4.0,NaN,NaN
4,Texas,TX,School,No,Sp Ed Co-Op,Other/alternative school,480000103621,54901200,Crosbyton CISD,Regular local school district,4800001,54901,7.0,Bi-annually,9/1/20,12/31/20,In-person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,0.0,0.0,NaN,NaN


In [3]:
virtual = pd.read_csv("../../data/raw_data/Texas_Schools_LearningModelData_Final.csv", thousands=',')

virtual['Charter'] = virtual['Charter'].replace({'Yes': 1, 'No': 0})

virtual = virtual[virtual['TimePeriodStart'].str.endswith(('21', '22'))]

# Convert the date columns to datetime format.
virtual['TimePeriodStart'] = pd.to_datetime(virtual['TimePeriodStart'])
virtual['TimePeriodEnd'] = pd.to_datetime(virtual['TimePeriodEnd'])

# Create a new column for year
virtual['YEAR'] = virtual['TimePeriodStart'].dt.year

# Fill in any missing values in LearningModel with 'InPerson'
virtual['LearningModel'] = virtual['LearningModel'].fillna('InPerson')

# Replace 'In-person' with 'InPerson'
virtual['LearningModel'] = virtual['LearningModel'].replace('In-person', 'InPerson')

# Calculate the number of days for each row
virtual['Days'] = (virtual['TimePeriodEnd'] - virtual['TimePeriodStart']).dt.days

# Group by School, Year, LearningModel, and Charter and sum the number of days
grouped = virtual.groupby(['StateAssignedSchoolID', 'YEAR', 'LearningModel', 'Charter'])['Days'].sum().reset_index()

# Pivot the data so we have separate columns for each learning model
pivot = grouped.pivot_table(index=['StateAssignedSchoolID', 'YEAR', 'Charter'], columns='LearningModel', values='Days', fill_value=0)

# Reset the index
pivot.reset_index(inplace=True)

# Calculate the total days in each year
pivot['TotalDays'] = pivot['Virtual'] + pivot['InPerson']

# Calculate the percentage of days that are virtual and hybrid for each year
pivot['VirtualPercent'] = pivot['Virtual'] / pivot['TotalDays']

# Manually create a 'HybridPercent' column and set it to zero
pivot['HybridPercent'] = 0  # This line added

# Calculate the score for each year
pivot['Score'] = pivot['VirtualPercent']

# Reset the column names after pivot
pivot.columns.name = None

In [4]:
common_ids = set(pivot["StateAssignedSchoolID"])

# Import Dropout Data

In [5]:
### Regressor Data
data21 = pd.read_excel("../../data/raw_data/campus-data-download-drop-2021.xlsx", thousands=',', sheet_name=2)
data19 = pd.read_excel("../../data/raw_data/campus-data-download-drop-1819.xlsx", thousands=',', sheet_name=2)
data18 = pd.read_excel("../../data/raw_data/campus-data-download-drop-1718.xlsx", thousands=',', sheet_name=2)
data17 = pd.read_excel("../../data/raw_data/campus-data-download-drop-1617.xlsx", thousands=',', sheet_name=2)
data16 = pd.read_excel("../../data/raw_data/campus-data-download-drop-1516.xlsx", thousands=',', sheet_name=2)
data15 = pd.read_excel("../../data/raw_data/campus-data-download-drop-1415.xlsx", thousands=',', sheet_name=2)

data18.head(5)

,CALC_FOR_STATE_ACCT,GRADESPAN,CAMPUS,CAMPNAME,DISTRICT,DISTNAME,COUNTY,CNTYNAME,REGION,REGNNAME,CAMP_ALLD,CAMP_ALLN,CAMP_ALLR,CAMP_AAD,CAMP_AAN,CAMP_AAR,CAMP_ASD,CAMP_ASN,CAMP_ASR,CAMP_HSD,CAMP_HSN,CAMP_HSR,CAMP_MUD,CAMP_MUN,CAMP_MUR,CAMP_NAD,CAMP_NAN,CAMP_NAR,CAMP_PID,CAMP_PIN,CAMP_PIR,CAMP_WHD,CAMP_WHN,CAMP_WHR,CAMP_ECND,CAMP_ECNN,CAMP_ECNR,CAMP_NECD,CAMP_NECN,CAMP_NECR,CAMP_FEMD,CAMP_FEMN,CAMP_FEMR,CAMP_MALD,CAMP_MALN,CAMP_MALR,CAMP_ATRD,CAMP_ATRN,CAMP_ATRR,CAMP_BED,CAMP_BEN,CAMP_BER,CAMP_CTED,CAMP_CTEN,CAMP_CTER,CAMP_DYSD,CAMP_DYSN,CAMP_DYSR,CAMP_FOSD,CAMP_FOSN,CAMP_FOSR,CAMP_GFTD,CAMP_GFTN,CAMP_GFTR,CAMP_HMLSD,CAMP_HMLSN,CAMP_HMLSR,CAMP_IMMD,CAMP_IMMN,CAMP_IMMR,CAMP_LEPD,CAMP_LEPN,CAMP_LEPR,CAMP_MIGD,CAMP_MIGN,CAMP_MIGR,CAMP_MILD,CAMP_MILN,CAMP_MILR,CAMP_OVRD,CAMP_OVRN,CAMP_OVRR,CAMP_SPED,CAMP_SPEN,CAMP_SPER,CAMP_TTLD,CAMP_TTLN,CAMP_TTLR
0,No,712,1902001,Cayuga H S,1902,Cayuga ISD,1,Anderson County,7,Kilgore,177,0,0.0,5,0,0.0,3,0,0.0,9,0,0.0,12,0,0.0,0,0,.,0,0,.,148,0,0.0,67,0,0.0,110,0,0.0,78,0,0.0,99,0,0.0,78,0,0.0,0,0,.,127,0,0.0,17,0,0.0,2,0,0.0,23,0,0.0,2,0,0.0,0,0,.,0,0,.,0,0,.,0,0,.,43,0,0.0,31,0,0.0,177,0,0.0
1,Yes,712,1902001,Cayuga H S,1902,Cayuga ISD,1,Anderson County,7,Kilgore,177,0,0.0,5,0,0.0,3,0,0.0,9,0,0.0,12,0,0.0,0,0,.,0,0,.,148,0,0.0,67,0,0.0,110,0,0.0,78,0,0.0,99,0,0.0,78,0,0.0,0,0,.,127,0,0.0,17,0,0.0,2,0,0.0,23,0,0.0,2,0,0.0,0,0,.,0,0,.,0,0,.,0,0,.,43,0,0.0,31,0,0.0,177,0,0.0
2,No,912,1902001,Cayuga H S,1902,Cayuga ISD,1,Anderson County,7,Kilgore,177,0,0.0,5,0,0.0,3,0,0.0,9,0,0.0,12,0,0.0,0,0,.,0,0,.,148,0,0.0,67,0,0.0,110,0,0.0,78,0,0.0,99,0,0.0,78,0,0.0,0,0,.,127,0,0.0,17,0,0.0,2,0,0.0,23,0,0.0,2,0,0.0,0,0,.,0,0,.,0,0,.,0,0,.,43,0,0.0,31,0,0.0,177,0,0.0
3,Yes,912,1902001,Cayuga H S,1902,Cayuga ISD,1,Anderson County,7,Kilgore,177,0,0.0,5,0,0.0,3,0,0.0,9,0,0.0,12,0,0.0,0,0,.,0,0,.,148,0,0.0,67,0,0.0,110,0,0.0,78,0,0.0,99,0,0.0,78,0,0.0,0,0,.,127,0,0.0,17,0,0.0,2,0,0.0,23,0,0.0,2,0,0.0,0,0,.,0,0,.,0,0,.,0,0,.,43,0,0.0,31,0,0.0,177,0,0.0
4,No,712,1902041,Cayuga Middle,1902,Cayuga ISD,1,Anderson County,7,Kilgore,108,0,0.0,3,0,0.0,1,0,0.0,7,0,0.0,9,0,0.0,0,0,.,0,0,.,88,0,0.0,52,0,0.0,56,0,0.0,50,0,0.0,58,0,0.0,46,0,0.0,0,0,.,0,0,.,5,0,0.0,0,0,.,7,0,0.0,0,0,.,0,0,.,0,0,.,0,0,.,1,0,0.0,34,0,0.0,17,0,0.0,108,0,0.0


In [6]:
data21.head(5)

,CALC_FOR_STATE_ACCT,GRADESPAN,CAMPUS,CAMPNAME,DISTRICT,DISTNAME,COUNTY,CNTYNAME,REGION,REGNNAME,CAMP_ALLD,CAMP_ALLN,CAMP_ALLR,CAMP_AAD,CAMP_AAN,CAMP_AAR,CAMP_ASD,CAMP_ASN,CAMP_ASR,CAMP_HSD,CAMP_HSN,CAMP_HSR,CAMP_MUD,CAMP_MUN,CAMP_MUR,CAMP_NAD,CAMP_NAN,CAMP_NAR,CAMP_PID,CAMP_PIN,CAMP_PIR,CAMP_WHD,CAMP_WHN,CAMP_WHR,CAMP_ECND,CAMP_ECNN,CAMP_ECNR,CAMP_NECD,CAMP_NECN,CAMP_NECR,CAMP_FEMD,CAMP_FEMN,CAMP_FEMR,CAMP_MALD,CAMP_MALN,CAMP_MALR,CAMP_ATRD,CAMP_ATRN,CAMP_ATRR,CAMP_BED,CAMP_BEN,CAMP_BER,CAMP_CTED,CAMP_CTEN,CAMP_CTER,CAMP_DYSD,CAMP_DYSN,CAMP_DYSR,CAMP_FOSD,CAMP_FOSN,CAMP_FOSR,CAMP_GFTD,CAMP_GFTN,CAMP_GFTR,CAMP_HMLSD,CAMP_HMLSN,CAMP_HMLSR,CAMP_IMMD,CAMP_IMMN,CAMP_IMMR,CAMP_LEPD,CAMP_LEPN,CAMP_LEPR,CAMP_MIGD,CAMP_MIGN,CAMP_MIGR,CAMP_FDMILD,CAMP_FDMILN,CAMP_FDMILR,CAMP_MILD,CAMP_MILN,CAMP_MILR,CAMP_OVRD,CAMP_OVRN,CAMP_OVRR,CAMP_SPED,CAMP_SPEN,CAMP_SPER,CAMP_SE5D,CAMP_SE5N,CAMP_SE5R,CAMP_TTLD,CAMP_TTLN,CAMP_TTLR
0,No,712,1902001,Cayuga H S,1902,Cayuga ISD,1,Anderson County,7,Kilgore,175,0,0.0,-3,-3,0.0,-1,-1,-1,14,0,0.0,8,0,0.0,0,0,.,0,0,.,146,0,0.0,64,0,0.0,111,0,0.0,82,0,0.0,93,0,0.0,68,0,0.0,-1,-1,-1,108,0,0.0,14,0,0.0,0,0,.,19,0,0.0,-1,-1,-1,0,0,.,-1,-1,-1,0,0,.,0,0,.,0,0,.,51,0,0.0,37,0,0.0,22,0,0.0,175,0,0.0
1,Yes,712,1902001,Cayuga H S,1902,Cayuga ISD,1,Anderson County,7,Kilgore,175,0,0.0,-3,-3,0.0,-1,-1,-1,14,0,0.0,8,0,0.0,0,0,.,0,0,.,146,0,0.0,64,0,0.0,111,0,0.0,82,0,0.0,93,0,0.0,68,0,0.0,-1,-1,-1,108,0,0.0,14,0,0.0,0,0,.,19,0,0.0,-1,-1,-1,0,0,.,-1,-1,-1,0,0,.,0,0,.,0,0,.,51,0,0.0,37,0,0.0,22,0,0.0,175,0,0.0
2,No,912,1902001,Cayuga H S,1902,Cayuga ISD,1,Anderson County,7,Kilgore,175,0,0.0,-3,-3,0.0,-1,-1,-1,14,0,0.0,8,0,0.0,0,0,.,0,0,.,146,0,0.0,64,0,0.0,111,0,0.0,82,0,0.0,93,0,0.0,68,0,0.0,-1,-1,-1,108,0,0.0,14,0,0.0,0,0,.,19,0,0.0,-1,-1,-1,0,0,.,-1,-1,-1,0,0,.,0,0,.,0,0,.,51,0,0.0,37,0,0.0,22,0,0.0,175,0,0.0
3,Yes,912,1902001,Cayuga H S,1902,Cayuga ISD,1,Anderson County,7,Kilgore,175,0,0.0,-3,-3,0.0,-1,-1,-1,14,0,0.0,8,0,0.0,0,0,.,0,0,.,146,0,0.0,64,0,0.0,111,0,0.0,82,0,0.0,93,0,0.0,68,0,0.0,-1,-1,-1,108,0,0.0,14,0,0.0,0,0,.,19,0,0.0,-1,-1,-1,0,0,.,-1,-1,-1,0,0,.,0,0,.,0,0,.,51,0,0.0,37,0,0.0,22,0,0.0,175,0,0.0
4,No,712,1902041,Cayuga Middle,1902,Cayuga ISD,1,Anderson County,7,Kilgore,93,0,0.0,-1,-1,-1,0,0,.,11,0,0.0,-1,-1,-1,0,0,.,0,0,.,77,0,0.0,43,0,0.0,50,0,0.0,42,0,0.0,51,0,0.0,39,0,0.0,0,0,.,0,0,.,6,0,0.0,0,0,.,7,0,0.0,0,0,.,0,0,.,0,0,.,0,0,.,-1,-1,-1,-1,-1,-1,23,0,0.0,11,0,0.0,14,0,0.0,93,0,0.0


In [7]:
common_ids = set(pivot["StateAssignedSchoolID"])

data = [data17, data18, data19, data21]
year = [2017, 2018, 2019, 2021]
# data = [data18, data19, data21]
# year = [2018, 2019, 2021]
calc_columns = ["CAMPUS","CAMP_ALLD","CAMP_ALLR","CAMP_AAD","CAMP_WHD","CAMP_HSD","CAMP_ASD","CAMP_ECND", "DISTRICT"]
final_columns = ["schoolcode","year","totalenrolled","dropout","lowincome","black","white","hispanic","asian", "district"]

for i in range(3):
    data[i].columns = data[i].columns.str.upper()

for i, (d, y) in enumerate(zip(data, year)):
    data[i].columns = data[i].columns.str.upper()
    data[i] = data[i][data[i]['GRADESPAN'] == 912]    
    data[i] = data[i][data[i]['CALC_FOR_STATE_ACCT'] == "Yes"]
    data[i] = data[i][calc_columns]
    mask = data[i].applymap(lambda x: '<' in str(x)).any(axis=1)
    data[i].replace(".", 0, inplace=True)
    data[i] = data[i][~mask]

# for i, (d, y) in enumerate(zip(data, year)):
# common_ids = common_ids & set(data[0]['CAMPUS']) & set(data[1]['CAMPUS']) & set(data[2]['CAMPUS'])

for i in range(len(data)):
    data[i] = data[i][data[i]['CAMPUS'].isin(common_ids)]

for i, (d, y) in enumerate(zip(data, year)):
    data[i].replace(0.0, 0, inplace=True)
    data[i]["schoolcode"] = data[i]["CAMPUS"].astype(float).astype(int)
    data[i]["totalenrolled"] = data[i]["CAMP_ALLD"].astype(float).astype(int)
    data[i]["dropout"] = data[i]["CAMP_ALLR"].astype(float).astype(int)
    print([min(data[i]["dropout"]), max(data[i]["dropout"])])
    data[i]["black"] = data[i]["CAMP_AAD"].astype(float).astype(int) / data[i]["CAMP_ALLD"].astype(float).astype(int)
    data[i]["white"] = data[i]["CAMP_WHD"].astype(float).astype(int) / data[i]["CAMP_ALLD"].astype(float).astype(int)
    data[i]["hispanic"] = data[i]["CAMP_HSD"].astype(float).astype(int) / data[i]["CAMP_ALLD"].astype(float).astype(int)
    data[i]["asian"] = data[i]["CAMP_ASD"].astype(float).astype(int) / data[i]["CAMP_ALLD"].astype(float).astype(int)
    data[i]["lowincome"] = data[i]["CAMP_ECND"].astype(float).astype(int) / data[i]["CAMP_ALLD"].astype(float).astype(int)
    data[i]["district"] = data[i]["DISTRICT"]
    
    
for i, (d, y) in enumerate(zip(data, year)):
    d['year'] = y
    data[i] = data[i][final_columns]

[0, 7]
[0, 15]
[0, 20]
[-1, 50]


/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipykernel_15073/3414176744.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = data[i].applymap(lambda x: '<' in str(x)).any(axis=1)
/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipykernel_15073/3414176744.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = data[i].applymap(lambda x: '<' in str(x)).any(axis=1)
/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipykernel_15073/3414176744.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = data[i].applymap(lambda x: '<' in str(x)).any(axis=1)
/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipykernel_15073/3414176744.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = data[i].applymap(lambda x: '<' in str(x)).any(axis=1)


In [8]:
pivot = pivot.drop(columns=['InPerson', 'Virtual', 'TotalDays'])
pivot

,StateAssignedSchoolID,YEAR,Charter,VirtualPercent,HybridPercent,Score
0,1902001,2021,0,0.0,0,0.0
1,1902041,2021,0,0.0,0,0.0
2,1902103,2021,0,0.0,0,0.0
3,1903001,2021,0,0.0,0,0.0
4,1903041,2021,0,0.0,0,0.0
...,...,...,...,...,...,...
8459,254901101,2021,0,0.0,0,0.0
8460,254901104,2021,0,0.0,0,0.0
8461,254901106,2021,0,0.0,0,0.0
8462,254902001,2021,0,1.0,0,1.0


In [9]:
# Concatenate all the dataframes
final_data = pd.concat(data)

In [10]:
problematic_cols = [
 'lowincome',
  'dropout',
 'totalenrolled',
      ]

final_data = final_data[~final_data[problematic_cols].lt(0).any(axis=1)]

In [11]:
# Merge dropout_df with final_data
final_data = pd.merge(final_data, pivot, left_on=['schoolcode', 'year'], right_on=['StateAssignedSchoolID', 'YEAR'], how='left')

In [12]:
final_data = final_data.fillna(0)
final_data = final_data.drop(columns=["StateAssignedSchoolID", "YEAR"])
# Rename multiple columns
final_data = final_data.rename(columns={               
                        'VirtualPercent': 'virtualper',    
                        'HybridPercent': 'hybridper',    
                        'Score': 'schoolmode',
                        'Charter': 'charter'
                                       })

final_cols = final_data.columns.tolist()
final_cols

['schoolcode',
 'year',
 'totalenrolled',
 'dropout',
 'lowincome',
 'black',
 'white',
 'hispanic',
 'asian',
 'district',
 'charter',
 'virtualper',
 'hybridper',
 'schoolmode']

In [13]:
final_cols = [
'schoolcode',
'district',
 'year',
 'charter',
 'dropout',
 'schoolmode',
 'virtualper',
 'hybridper',
 'totalenrolled',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'asian',
      ]

multi_cols = [
'schoolcode',
'district',
 'year',
 'charter',
 'dropout',
 'totalenrolled',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'asian',
      ]

In [14]:
final_data = final_data[final_cols]
for column in multi_cols[5:]:
    if column != "totalenrolled":
        final_data[column] = final_data[column] * 100

In [15]:
problematic_cols = [
 'white',
 'black',
 'hispanic',
 'asian',
      ]

rounding_cols = [
 'schoolmode',
 'virtualper',
 'hybridper',
 'totalenrolled',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'asian',
      ]

final_data[problematic_cols] = final_data[problematic_cols].abs()
final_data[rounding_cols] = final_data[rounding_cols].round(4)

In [16]:
# print(final_data["dropout"].value_counts().to_dict())
# print(final_data["dropout"].value_counts().to_dict())
export_dataframe = final_data

export_dataframe["schoolcode"] = export_dataframe["schoolcode"].astype(int).astype(str)

export_dataframe["state"] = "tx"
export_dataframe["schoolcode"] = export_dataframe["schoolcode"].str.zfill(9)
export_dataframe["countycode"] = export_dataframe["schoolcode"].str[:3].astype(str)
export_dataframe["districtcode"] = export_dataframe["schoolcode"].str[:6].astype(str)
export_dataframe['districtcode'] = export_dataframe['districtcode'].astype(str)
export_dataframe['countycode'] = export_dataframe['countycode'].astype(str)

export_dataframe["schoolcode"] = export_dataframe["schoolcode"] + "tx"
export_dataframe["districtcode"] = export_dataframe["districtcode"] + "tx"
export_dataframe["countycode"] = export_dataframe["countycode"] + "tx"

final_cols = [
'schoolcode',
"districtcode",
"countycode",
 'year',
 'charter',
 'dropout',
 'schoolmode',
 'virtualper',
 'hybridper',
 'totalenrolled',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'state'
      ]

export_dataframe = export_dataframe[final_cols]
export_dataframe.head(10)

# Identify the unique years in the dataset.
unique_years = set(export_dataframe['year'])

# Group the data by ENTITY_CD and get the unique years for each group.
entity_groups = export_dataframe.groupby('schoolcode')['year'].unique().reset_index()

# Find the ENTITY_CD values which have all the unique years.
valid_entity_cd = entity_groups[entity_groups['year'].apply(lambda x: set(x) == unique_years)]['schoolcode']

# Filter the main data for these ENTITY_CD values.
export_dataframe = export_dataframe[export_dataframe['schoolcode'].isin(valid_entity_cd)]

# Export Texas Data

In [17]:
export_dataframe.to_csv("../final_data_component/final_data_tx_dropout.csv")

In [18]:
problematic_cols = [
 'white',
 'black',
 'hispanic',
 'lowincome',
 'dropout',
 'totalenrolled'
      ]

export_dataframe[export_dataframe[problematic_cols].lt(0).any(axis=1)]

,schoolcode,districtcode,countycode,year,charter,dropout,schoolmode,virtualper,hybridper,totalenrolled,lowincome,white,black,hispanic,state


In [19]:
export_dataframe.describe()

,year,charter,dropout,schoolmode,virtualper,hybridper,totalenrolled,lowincome,white,black,hispanic
count,1480.00000,1480.000000,1480.000000,1480.000000,1480.000000,1480.0,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000
mean,2018.75000,0.043919,0.039189,0.031081,0.031081,0.0,220.112162,54.414956,46.575221,6.401077,41.204748
std,1.47952,0.204984,0.535253,0.173596,0.173596,0.0,251.109577,22.575962,29.892065,11.209590,28.849466
min,2017.00000,0.000000,0.000000,0.000000,0.000000,0.0,5.000000,0.000000,0.000000,0.000000,0.000000
25%,2017.75000,0.000000,0.000000,0.000000,0.000000,0.0,67.000000,40.211675,17.090900,0.000000,16.314650
50%,2018.50000,0.000000,0.000000,0.000000,0.000000,0.0,140.000000,54.853100,50.384500,1.785700,35.250250
75%,2019.50000,0.000000,0.000000,0.000000,0.000000,0.0,313.000000,70.982775,73.686625,7.143625,60.782325
max,2021.00000,1.000000,16.000000,1.000000,1.000000,0.0,2236.000000,100.000000,100.000000,84.304900,100.000000
